In [8]:
import pyspark
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .master("local[*]") \
    .appName('test') \
    .getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/06/10 06:14:26 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
23/06/10 06:14:27 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [45]:
spark.sparkContext.getConf().getAll()

[('spark.driver.extraJavaOptions',
  '-XX:+IgnoreUnrecognizedVMOptions --add-opens=java.base/java.lang=ALL-UNNAMED --add-opens=java.base/java.lang.invoke=ALL-UNNAMED --add-opens=java.base/java.lang.reflect=ALL-UNNAMED --add-opens=java.base/java.io=ALL-UNNAMED --add-opens=java.base/java.net=ALL-UNNAMED --add-opens=java.base/java.nio=ALL-UNNAMED --add-opens=java.base/java.util=ALL-UNNAMED --add-opens=java.base/java.util.concurrent=ALL-UNNAMED --add-opens=java.base/java.util.concurrent.atomic=ALL-UNNAMED --add-opens=java.base/sun.nio.ch=ALL-UNNAMED --add-opens=java.base/sun.nio.cs=ALL-UNNAMED --add-opens=java.base/sun.security.action=ALL-UNNAMED --add-opens=java.base/sun.util.calendar=ALL-UNNAMED --add-opens=java.security.jgss/sun.security.krb5=ALL-UNNAMED'),
 ('spark.app.name', 'test'),
 ('spark.executor.id', 'driver'),
 ('spark.app.startTime', '1686377666537'),
 ('spark.driver.port', '37705'),
 ('spark.sql.warehouse.dir',
  'file:/home/deepak/data-engineering-zoomcamp/week_5_batch_proce

In [9]:
spark.sql("select 1")

DataFrame[1: int]

In [10]:
df_green = spark.read.parquet('data/pq/green/*/*')

In [11]:
df_green.registerTempTable('green')

/home/deepak/spark/spark-3.3.2-bin-hadoop3/python/pyspark/sql/dataframe.py:229: FutureWarning: Deprecated in 2.0, use createOrReplaceTempView instead.
  warnings.warn("Deprecated in 2.0, use createOrReplaceTempView instead.", FutureWarning)


In [12]:
df_green_revenue = spark.sql("""
SELECT 
    date_trunc('hour', lpep_pickup_datetime) AS hour, 
    PULocationID AS zone,

    SUM(total_amount) AS amount,
    COUNT(1) AS number_records
FROM
    green
WHERE
    lpep_pickup_datetime >= '2020-01-01 00:00:00'
GROUP BY
    1, 2
""")

In [13]:
df_green_revenue \
    .repartition(20) \
    .write.parquet('data/report/revenue/green', mode='overwrite')

In [14]:
df_yellow = spark.read.parquet('data/pq/yellow/*/*')
df_yellow.registerTempTable('yellow')

/home/deepak/spark/spark-3.3.2-bin-hadoop3/python/pyspark/sql/dataframe.py:229: FutureWarning: Deprecated in 2.0, use createOrReplaceTempView instead.
  warnings.warn("Deprecated in 2.0, use createOrReplaceTempView instead.", FutureWarning)


In [17]:
df_yellow.columns

['VendorID',
 'lpep_pickup_datetime',
 'lpep_dropoff_datetime',
 'store_and_fwd_flag',
 'RatecodeID',
 'PULocationID',
 'DOLocationID',
 'passenger_count',
 'trip_distance',
 'fare_amount',
 'extra',
 'mta_tax',
 'tip_amount',
 'tolls_amount',
 'ehail_fee',
 'improvement_surcharge',
 'total_amount',
 'payment_type',
 'trip_type',
 'congestion_surcharge']

In [18]:
df_yellow_revenue = spark.sql("""
SELECT 
    date_trunc('hour', lpep_pickup_datetime) AS hour, 
    PULocationID AS zone,

    SUM(total_amount) AS amount,
    COUNT(1) AS number_records
FROM
    yellow
WHERE
    lpep_pickup_datetime >= '2020-01-01 00:00:00'
GROUP BY
    1, 2
""")

In [19]:
df_yellow_revenue \
    .repartition(20) \
    .write.parquet('data/report/revenue/yellow', mode='overwrite')

In [20]:
df_green_revenue = spark.read.parquet('data/report/revenue/green')
df_yellow_revenue = spark.read.parquet('data/report/revenue/yellow')

In [21]:
df_green_revenue_tmp = df_green_revenue \
    .withColumnRenamed('amount', 'green_amount') \
    .withColumnRenamed('number_records', 'green_number_records')

df_yellow_revenue_tmp = df_yellow_revenue \
    .withColumnRenamed('amount', 'yellow_amount') \
    .withColumnRenamed('number_records', 'yellow_number_records')

In [22]:
#final_df_name = <first_df>.join(<second_df>, on='col' OR ['col1', 'col2'], how= <'outer', 'left', 'right' etc.>) \
#                    .join(<third>, on='col' OR OR ['col1', 'col2'], how= <'outer', 'left', 'right' etc.>)

df_join = df_green_revenue_tmp.join(df_yellow_revenue_tmp, on=['hour', 'zone'], how='outer')

In [23]:
df_join.show(10)

+-------------------+----+------------------+--------------------+-------------+---------------------+
|               hour|zone|      green_amount|green_number_records|yellow_amount|yellow_number_records|
+-------------------+----+------------------+--------------------+-------------+---------------------+
|2020-01-01 00:00:00|   7| 769.7299999999996|                  45|         null|                 null|
|2020-01-01 00:00:00|  37|            175.67|                   6|         null|                 null|
|2020-01-01 00:00:00|  40|168.97999999999996|                   8|         null|                 null|
|2020-01-01 00:00:00|  47|              13.3|                   1|         null|                 null|
|2020-01-01 00:00:00|  51|              17.8|                   2|         null|                 null|
|2020-01-01 00:00:00|  62|             15.95|                   1|         null|                 null|
|2020-01-01 00:00:00|  74|317.09000000000015|                  24|       

In [24]:
df_join.write.parquet('data/report/revenue/total', mode='overwrite')

In [25]:
df_join = spark.read.parquet('data/report/revenue/total')

In [26]:
df_join

DataFrame[hour: timestamp, zone: int, green_amount: double, green_number_records: bigint, yellow_amount: double, yellow_number_records: bigint]

In [40]:
df_zones = spark.read.parquet('zones/')

In [41]:
# broadcasting .the smaller dataset is broadcasted to all the executors 
df_result = df_join.join(df_zones, df_join.zone == df_zones.LocationID)

In [42]:
df_result.drop('LocationID', 'zone').write.parquet('tmp/revenue-zones')